In [5]:
from datasets import load_dataset
import tensorflow as tf
import pynvml

In [ ]:
#dataset = load_dataset('reddit', download_mode= "reuse_cache_if_exists")

Using custom data configuration default


In [13]:
dataset

DatasetDict({
    train: Dataset({
        features: ['author', 'body', 'normalizedBody', 'subreddit', 'subreddit_id', 'id', 'content', 'summary'],
        num_rows: 3848330
    })
})

In [19]:
dataset['train'][4]
#we only need the columns 'content' and 'summary' for training

{'author': 'NuffZetPand0ra',
 'body': "You are talking about the Charsi imbue, right? Or a cube upgrade?\nIf we are talking Charsi imbue, you can only imbue WHITE items. This includes superior, but they will not neccesarily be superior after imbuing (they get random base-modifications). Bloodfist and Gorefoot are both uniques (gold), and therefore not eligible for imbuing.\nWhen you imbue, the item level matters (the item level is hidden). The item is the same level as the monster who dropped it. That means, that the higher level the monster who dropped it, the more stats is available on that item. It is important to note that an item doesn't neccesarily use all it's stat potential. This means that the same item dropped in a1 and a2 can has the possibility of some very different outcomes.\nAfter the imbue, the item can be as good as if the monster itself had dropped a rare (yellow) item. Imbued weapons will always turn out as rare items.\nTo answer your question, you should just progre

In [20]:
ds = dataset.remove_columns(['author',
  'body',
  'normalizedBody',
  'subreddit',
  'subreddit_id',
  'id'])
ds

DatasetDict({
    train: Dataset({
        features: ['content', 'summary'],
        num_rows: 3848330
    })
})

#we'll shuffle the dataset and keep one million of examples. It's a very large dataset and we need to find  balance between efficient training and computational resources. In the paper "RL4LM" the authors used CNN daily dataset for fine-tuning an abstractive summarization model. The dataset contained 311K examples with 300K being train-validation and 11K being test. We'll keep 400K examples from our dataset for fine-tuning our model.


In [22]:
#first we'll shuffle dataset to ensure the randomness of examples
shuffled_ds = ds.shuffle(seed=42)

Loading cached shuffled indices for dataset at C:\Users\VR\.cache\huggingface\datasets\reddit\default\1.0.0\98ba5abea674d3178f7588aa6518a5510dc0c6fa8176d9653a3546d5afcb3969\cache-1bd07cb9828bf1ee.arrow


In [33]:
ds['train'].info

DatasetInfo(description='\nThis corpus contains preprocessed posts from the Reddit dataset.\nThe dataset consists of 3,848,330 posts with an average length of 270 words for content,\nand 28 words for the summary.\n\nFeatures includes strings: author, body, normalizedBody, content, summary, subreddit, subreddit_id.\nContent is used as document and summary is used as summary.\n', citation='\n@inproceedings{volske-etal-2017-tl,\n    title = {TL;DR: Mining {R}eddit to Learn Automatic Summarization},\n    author = {V{"o}lske, Michael  and Potthast, Martin  and Syed, Shahbaz  and Stein, Benno},\n    booktitle = {Proceedings of the Workshop on New Frontiers in Summarization},\n    month = {sep},\n    year = {2017},\n    address = {Copenhagen, Denmark},\n    publisher = {Association for Computational Linguistics},\n    url = {https://www.aclweb.org/anthology/W17-4508},\n    doi = {10.18653/v1/W17-4508},\n    pages = {59--63},\n    abstract = {Recent advances in automatic text summarization hav

In [40]:
raw_ds = shuffled_ds['train'].select(range(400000))
raw_ds

Dataset({
    features: ['content', 'summary'],
    num_rows: 400000
})

In [41]:
#we'll save the raw dataset to our local computer to avoid reloading the full dataset from Huggingface
raw_ds.save_to_disk('C:/Users/VR/.cache/huggingface/datasets')

Flattening the indices: 100%|██████████| 400/400 [00:04<00:00, 90.41ba/s]


In [43]:
raw_ds = load_from_disk('C:/Users/VR/.cache/huggingface/datasets')
raw_ds

Dataset({
    features: ['content', 'summary'],
    num_rows: 400000
})

In [44]:
raw_ds=raw_ds.train_test_split(test_size = 0.1)
raw_ds

DatasetDict({
    train: Dataset({
        features: ['content', 'summary'],
        num_rows: 360000
    })
    test: Dataset({
        features: ['content', 'summary'],
        num_rows: 40000
    })
})

In [1]:
from pynvml import *
def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

In [2]:
print_gpu_utilization()

GPU memory occupied: 663 MB.
